In [29]:
import numpy as np

class Dense_Layer:
    def __init__(self, n_inputs, n_neurons, activation="linear"):
        self.n_inputs = n_inputs
        self.n_neurons = n_neurons
        self.activation_name = activation.lower()
        self.weights = np.zeros((n_inputs, n_neurons))
        self.biases = np.zeros((1, n_neurons))
        self.inputs = None
        self.z = None
        self.a = None

    def set_inputs_weights(self, inputs, weights=None, biases=None):
        self.inputs = np.array(inputs, dtype=float).reshape(1, -1)
        if weights is not None:
            self.weights = np.array(weights, dtype=float).reshape(self.n_inputs, self.n_neurons)
        if biases is not None:
            self.biases = np.array(biases, dtype=float).reshape(1, self.n_neurons)

    def weighted_sum(self):
        self.z = self.inputs.dot(self.weights) + self.biases
        return self.z

    def activate(self):
        if self.activation_name == "relu":
            self.a = np.maximum(0, self.z)
        elif self.activation_name == "sigmoid":
            self.a = 1 / (1 + np.exp(-self.z))
        elif self.activation_name == "softmax":
            z_stable = self.z - np.max(self.z, axis=1, keepdims=True)
            exp_z = np.exp(z_stable)
            self.a = exp_z / np.sum(exp_z, axis=1, keepdims=True)
        else:
            self.a = self.z
        return self.a

    def forward(self, inputs=None):
        if inputs is not None:
            self.inputs = np.array(inputs, dtype=float).reshape(1, -1)
        z = self.weighted_sum()
        a = self.activate()
        return z, a

    @staticmethod
    def loss(y_pred, y_true):
        y_pred = np.array(y_pred, dtype=float)
        y_true = np.array(y_true, dtype=float)
        if y_pred.ndim == 1:
            y_pred = y_pred.reshape(1, -1)
        if y_true.ndim == 1:
            y_true = y_true.reshape(1, -1)
        epsilon = 1e-15
        y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
        if y_pred.shape[1] == 1:
            return -np.sum(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred)) / y_true.shape[0]
        else:
            return -np.sum(y_true * np.log(y_pred)) / y_true.shape[0]

In [31]:

# 1) Iris Dataset Example

print("=== IRIS DATASET ===")

X_iris = [5.1, 3.5, 1.4, 0.2]
target_iris = [0.7, 0.2, 0.1]

# Layer 1
W1_iris = [[0.2, 0.5, -0.3],
           [0.1, -0.2, 0.4],
           [-0.4, 0.3, 0.2],
           [0.6, -0.1, 0.5]]
B1_iris = [3.0, -2.1, 0.6]
layer1_iris = Dense_Layer(4, 3, 'relu')
layer1_iris.set_inputs_weights(X_iris, W1_iris, B1_iris)
z1, a1 = layer1_iris.forward()
print("=== FIRST HIDDEN LAYER ===")
print("Weighted Sum:", z1.flatten())
print("After ReLU:", a1.flatten())

# Layer 2
W2_iris = [[0.3, -0.5],
           [0.7, 0.2],
           [-0.6, 0.4]]
B2_iris = [4.3, 6.4]
layer2_iris = Dense_Layer(3, 2, 'sigmoid')
layer2_iris.set_inputs_weights(a1, W2_iris, B2_iris)
z2, a2 = layer2_iris.forward()
print("\n=== SECOND HIDDEN LAYER ===")
print("Weighted Sum:", z2.flatten())
print("After Sigmoid:", a2.flatten())

# Layer 3
W3_iris = [[0.5, -0.3, 0.8],
           [-0.2, 0.6, -0.4]]
B3_iris = [-1.5, 2.1, -3.3]
layer3_iris = Dense_Layer(2, 3, 'softmax')
layer3_iris.set_inputs_weights(a2, W3_iris, B3_iris)
z3, a3 = layer3_iris.forward()
print("\n=== THIRD HIDDEN LAYER ===")
print("Weighted Sum:", z3.flatten())
print("After Softmax:", a3.flatten())

loss_iris = Dense_Layer.loss(a3, target_iris)
print("\nHidden Layer 2 (Output) Loss:", loss_iris)




=== IRIS DATASET ===
=== FIRST HIDDEN LAYER ===
Weighted Sum: [3.93 0.15 0.85]
After ReLU: [3.93 0.15 0.85]

=== SECOND HIDDEN LAYER ===
Weighted Sum: [5.074 4.805]
After Sigmoid: [0.99378157 0.99187781]

=== THIRD HIDDEN LAYER ===
Weighted Sum: [-1.20148478  2.39699221 -2.90172587]
After Softmax: [0.0265075  0.96865119 0.00484132]

Hidden Layer 2 (Output) Loss: 3.080656405230887


In [32]:

# 2) Breast Cancer Dataset Example

print("\n=== BREAST CANCER DATASET ===")

X_cancer = [14.1, 20.3, 0.095]
target_cancer = [1]

# Layer 1
W1_c = [[0.5, -0.3, 0.8],
        [0.2, 0.4, -0.6],
        [-0.7, 0.9, 0.1]]
B1_c = [0.3, -0.5, 0.6]
layer1_c = Dense_Layer(3, 3, 'relu')
layer1_c.set_inputs_weights(X_cancer, W1_c, B1_c)
z1_c, a1_c = layer1_c.forward()
print("=== FIRST HIDDEN LAYER ===")
print("Weighted Sum:", z1_c.flatten())
print("After ReLU:", a1_c.flatten())

# Layer 2
W2_c = [[0.6, -0.2],
        [0.4, -0.3],
        [0.5, 0.7]]
B2_c = [0.1, -0.8]
layer2_c = Dense_Layer(3, 2, 'sigmoid')
layer2_c.set_inputs_weights(a1_c, W2_c, B2_c)
z2_c, a2_c = layer2_c.forward()
print("\n=== SECOND HIDDEN LAYER ===")
print("Weighted Sum:", z2_c.flatten())
print("After Sigmoid:", a2_c.flatten())

# Layer 3
W3_c = [[0.7],
        [-0.5]]
B3_c = [0.2]
layer3_c = Dense_Layer(2, 1, 'sigmoid')
layer3_c.set_inputs_weights(a2_c, W3_c, B3_c)
z3_c, a3_c = layer3_c.forward()
print("\n=== THIRD HIDDEN LAYER ===")
print("Weighted Sum:", z3_c.flatten())
print("After Sigmoid:", a3_c.flatten())

loss_cancer = Dense_Layer.loss(a3_c, target_cancer)
print("\nHidden Layer 2 (Output) Loss:", loss_cancer)


=== BREAST CANCER DATASET ===
=== FIRST HIDDEN LAYER ===
Weighted Sum: [11.3435  3.4755 -0.2905]
After ReLU: [11.3435  3.4755  0.    ]

=== SECOND HIDDEN LAYER ===
Weighted Sum: [ 8.2963  -4.11135]
After Sigmoid: [0.99975062 0.01612148]

=== THIRD HIDDEN LAYER ===
Weighted Sum: [0.8917647]
After Sigmoid: [0.70925421]

Hidden Layer 2 (Output) Loss: 0.343541269530652
